In [7]:
import os
import pandas as pd

def merge_files(raw_folder, output_file):
    electricity_folder = os.path.join(raw_folder, "electricity_raw_data")
    weather_folder = os.path.join(raw_folder, "weather_raw_data")

    electricity_files = [os.path.join(electricity_folder, f) for f in os.listdir(electricity_folder) if f.endswith('.json')]
    weather_files = [os.path.join(weather_folder, f) for f in os.listdir(weather_folder) if f.endswith('.csv')]

    electricity_data = []
    weather_data = []

    for file in electricity_files:
        with open(file, 'r') as f:
            data = pd.json_normalize(pd.read_json(f)['response']['data'])
            data['datetime'] = pd.to_datetime(data['period'], format='%Y-%m-%dT%H')
            data.drop(columns=['period'], inplace=True)
            electricity_data.append(data)

    for file in weather_files:
        data = pd.read_csv(file)
        data.rename(columns={'date': 'datetime'}, inplace=True)
        data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
        weather_data.append(data)

    if electricity_data and weather_data:
        electricity_df = pd.concat(electricity_data, ignore_index=True)
        weather_df = pd.concat(weather_data, ignore_index=True)

        if weather_df['datetime'].dtype == 'object':
            weather_df['datetime'] = pd.to_datetime(weather_df['datetime'], errors='coerce')

        weather_df['datetime'] = weather_df['datetime'].dropna().dt.tz_localize(None)

        merged_df = pd.merge(electricity_df, weather_df, on='datetime', how='inner')
        merged_df.to_csv(output_file, index=False)
        print(f"Merged file saved at: {output_file}")
    else:
        print("No valid files found in raw folder.")

# Example usage
raw_folder_path = "raw"  # Update this to the actual path
output_file_path = "merged_data.csv"
merge_files(raw_folder_path, output_file_path)

Merged file saved at: merged_data.csv
